### Objetivo
Neste caderno vamos criar um código que verifica se pessoas com alto patrimônio estão recebendo o auxílio emergencial. É simples! Vamos, primeiramente, levantar os dados das declarações de bens feitas por candidatos nas eleições de 2020. Em seguida, selecionaremos os candidatos com patrimônio declarado acima de R$ 1 milhão. Aí usaremos a API do Portal da Transparência para verificar se eles estão entre os beneficiários do Coronavoucher.

### Pacotes utilizados

In [37]:
import pandas as pd ### Pacote para análise e transformação de tabelas de dados
import requests ### Pacote para requisição de dados na web
import time ### Pacote para delimitar intervalo de tempo no looping


### 1. Carregando dados do TSE

##### -> Bens dos candidatos
Fonte: https://cdn.tse.jus.br/estatistica/sead/odsele/bem_candidato/bem_candidato_2020.zip

In [38]:
df_bens = pd.read_csv('../_dados/bem_candidato_2020_BRASIL.csv', encoding = 'latin1', sep = ';')

In [39]:
print('Colunas - df_bens: ')
df_bens.columns

Colunas - df_bens: 


Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO', 'SG_UF',
       'SG_UE', 'NM_UE', 'SQ_CANDIDATO', 'NR_ORDEM_CANDIDATO',
       'CD_TIPO_BEM_CANDIDATO', 'DS_TIPO_BEM_CANDIDATO', 'DS_BEM_CANDIDATO',
       'VR_BEM_CANDIDATO', 'DT_ULTIMA_ATUALIZACAO', 'HH_ULTIMA_ATUALIZACAO'],
      dtype='object')

##### -> Dados cadastrais dos candidatos
Fonte: https://cdn.tse.jus.br/estatistica/sead/odsele/consulta_cand/consulta_cand_2020.zip

In [40]:
df_cadastro = pd.read_csv('../_dados/consulta_cand_2020_BRASIL.csv', encoding = 'latin1', sep = ';')

In [41]:
print('Colunas - df_cadastro: ')
df_cadastro.columns

Colunas - df_cadastro: 


Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'NR_TURNO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO',
       'TP_ABRANGENCIA', 'SG_UF', 'SG_UE', 'NM_UE', 'CD_CARGO', 'DS_CARGO',
       'SQ_CANDIDATO', 'NR_CANDIDATO', 'NM_CANDIDATO', 'NM_URNA_CANDIDATO',
       'NM_SOCIAL_CANDIDATO', 'NR_CPF_CANDIDATO', 'NM_EMAIL',
       'CD_SITUACAO_CANDIDATURA', 'DS_SITUACAO_CANDIDATURA',
       'CD_DETALHE_SITUACAO_CAND', 'DS_DETALHE_SITUACAO_CAND', 'TP_AGREMIACAO',
       'NR_PARTIDO', 'SG_PARTIDO', 'NM_PARTIDO', 'SQ_COLIGACAO',
       'NM_COLIGACAO', 'DS_COMPOSICAO_COLIGACAO', 'CD_NACIONALIDADE',
       'DS_NACIONALIDADE', 'SG_UF_NASCIMENTO', 'CD_MUNICIPIO_NASCIMENTO',
       'NM_MUNICIPIO_NASCIMENTO', 'DT_NASCIMENTO', 'NR_IDADE_DATA_POSSE',
       'NR_TITULO_ELEITORAL_CANDIDATO', 'CD_GENERO', 'DS_GENERO',
       'CD_GRAU_INSTRUCAO', 'DS_GRAU_INSTRUCAO', 'CD_ESTADO_CIVIL',
       'DS_ESTADO_CIVIL', 'CD_COR_RACA', 'DS_COR_RACA', 'CD_OCUPACAO',
       '

### 2. Identificando milionários

In [42]:
# Tratando campo de valor dos bens
df_bens['VR_BEM_CANDIDATO'] = df_bens['VR_BEM_CANDIDATO'].apply(lambda x: float(x.replace(',', '.')))

In [43]:
# Somando valor dos bens por candidato
df_bens_agregado = df_bens[['SQ_CANDIDATO', 'VR_BEM_CANDIDATO']].groupby(['SQ_CANDIDATO']).sum().reset_index()

In [44]:
# Trazendo o CPF do candidato
df_bens_agregado['SQ_CANDIDATO'] = df_bens_agregado['SQ_CANDIDATO'].apply(lambda x: int(x))
df_bens_agregado = df_bens_agregado.merge(df_cadastro[['SQ_CANDIDATO', 'NR_CPF_CANDIDATO', 'NM_CANDIDATO', 'SG_PARTIDO', 'SG_UF', 'NM_UE', 'DS_CARGO']], how = 'inner', on = 'SQ_CANDIDATO')
print(len(df_bens_agregado))
df_bens_agregado.head()

340452


,SQ_CANDIDATO,VR_BEM_CANDIDATO,NR_CPF_CANDIDATO,NM_CANDIDATO,SG_PARTIDO,SG_UF,NM_UE,DS_CARGO
0,10000641345,319500.0,78506735220,GILSON DA COSTA DIAS,PSD,AC,ASSIS BRASIL,VEREADOR
1,10000641349,179000.0,19738595215,MANOEL DE OLIVEIRA SOUZA,PSD,AC,ASSIS BRASIL,VEREADOR
2,10000641351,7000.0,52559505215,LUIZ ABEL PEREIRA JAMINAWA,PSD,AC,ASSIS BRASIL,VEREADOR
3,10000641352,150000.0,67465994268,ANA CLAUDIA DA SILVA GONCALVES DE MORAES,PSD,AC,ASSIS BRASIL,VEREADOR
4,10000641353,150000.0,69403708204,IVANEIDE DA SILVA ARAUJO,PSD,AC,ASSIS BRASIL,VEREADOR


In [45]:
# Filtrando milionários
df_milionarios = df_bens_agregado[df_bens_agregado['VR_BEM_CANDIDATO'] >= 1000000]
df_milionarios.sample(5)

,SQ_CANDIDATO,VR_BEM_CANDIDATO,NR_CPF_CANDIDATO,NM_CANDIDATO,SG_PARTIDO,SG_UF,NM_UE,DS_CARGO
260423,240000752606,1276987.17,47746017991,JOSÉ HENRIQUE REMPEL HENTZ,PT,SC,PINHALZINHO,VEREADOR
112845,130000855128,2043000.00,88152014672,CARLOS ROBERTO MENDES,PP,MG,CATAS ALTAS DA NORUEGA,VEREADOR
271250,240001138198,1000000.00,89774183991,TÂNIA MARIA MARCELINO,PSD,SC,BIGUAÇU,VEREADOR
314325,250001111226,1695000.00,7781332814,NEY VAZ PINTO LYRA,PSDB,SP,BERTIOGA,VEREADOR
147018,130001235138,1280000.00,84025085615,GERALDO FLAVIO DE MACEDO SOARES,MDB,MG,MONTALVÂNIA,PREFEITO


In [46]:
print('Número de Milionários - Eleições 20020:')
len(df_milionarios)

Número de Milionários - Eleições 20020:


13510

### 3. Acessando API do Portal da Transparência para verificar se o ricaço está recebendo o Coronavoucher
Para obter credencial na API: http://portaldatransparencia.gov.br/api-de-dados/cadastrar-email

#### -> Primeiro, vamos só verificar como voltam as requisições do coronavoucher

In [47]:
exemplo_cpf_recebe = '70068684240'
exemplo_cpf_nao_recebe = '15326950120'

# Aqui são os parâmetros da conexão, onde informamos nossa credencial de acesso
headers = {
    'Accept': "application/json",
    'chave-api-dados': '##### sua credencial aqui #####',
}

# Aqui, os parâmetros da consulta. Primeiro, vamos ver um caso em que o cpf recebe auxílio emergencial
params = (
        ('codigoBeneficiario', exemplo_cpf_recebe),
        ('pagina', '1'),
    )

# Aqui, a resposta dessa requisição
response = requests.get('http://www.transparencia.gov.br/api-de-dados/auxilio-emergencial-por-cpf-ou-nis', headers=headers, params=params)
print('Exemplo positivo')
print(response.json())
print('----------')

# Agora, vejamos a resposta de uma requisição onde o cpf não recebe o auxílio
params = (
        ('codigoBeneficiario', exemplo_cpf_nao_recebe),
        ('pagina', '1'),
    )

response = requests.get('http://www.transparencia.gov.br/api-de-dados/auxilio-emergencial-por-cpf-ou-nis', headers=headers, params=params)
print('Exemplo negativo')
print(response.json())
print('----------')

print('Veja nos 2 exemplos acima que, quando o cpf é beneficiário do coronavoucher, a requisição retorna com várias informações sobre o pagamento do benefício.')
print('Já quando não há recebimento, a resposta é uma lista vazia.')

Exemplo positivo
[{'id': 835685928, 'mesDisponibilizacao': '04/2020', 'beneficiario': {'nis': '00000000000', 'nome': 'NESTOR DIEGO VACA PONZ', 'multiploCadastro': False, 'cpfFormatado': '***.686.842-**'}, 'responsavelAuxilioEmergencial': {'nis': '-2', 'nome': 'Não se aplica', 'nomeSemAcento': 'Nao se aplica', 'cpfFormatado': ''}, 'municipio': {'codigoIBGE': '1200104', 'nomeIBGE': 'BRASILÉIA', 'nomeIBGEsemAcento': 'BRASILEIA', 'pais': 'BRASIL', 'uf': {'sigla': 'AC', 'nome': 'ACRE'}}, 'valor': 600.0}]
----------
Exemplo negativo
[]
----------
Veja nos 2 exemplos acima que, quando o cpf é beneficiário do coronavoucher, a requisição retorna com várias informações sobre o pagamento do benefício.
Já quando não há recebimento, a resposta é uma lista vazia.


#### -> Agora sim, vamos verificar nossos milionários

In [48]:
lista_milionarios = list(df_milionarios['NR_CPF_CANDIDATO'])

In [49]:
headers = {
    'Accept': "application/json",
    'chave-api-dados': '##### sua credencial aqui #####',
}

# Testando abaixo se cada CPF da minha lista de milionários consta como beneficiário do Coronavoucher.
respostas = []
for cpf in lista_milionarios[:20]: # Para economizar tempo na oficina, estou restringindo a consulta aos 20 primeiros.
    time.sleep(2.01)
    cpf_ = '0'*(11-len(str(cpf))) + str(cpf)
    params = (
        ('codigoBeneficiario', cpf_),
        ('pagina', '1'),
    )

    response = requests.get('http://www.transparencia.gov.br/api-de-dados/auxilio-emergencial-por-cpf-ou-nis', headers=headers, params=params)
    if len(response.json()) > 0:
        teste = 1
    else:
        teste = 0
        
    respostas.append([cpf, teste])

    print(cpf, teste)


30875978215 0
35936398234 0
70068684240 1
80791581268 1
31207642800 0
72816082253 0
19672705204 0
47655640963 0
81428197249 1
65406915720 0
34979824234 0
47837250206 1
55338615720 0
19631642291 0
21711615234 0
50791524272 0
50810006200 0
35954590249 0
42062160291 0
43403956253 0


In [50]:
respostas

[[30875978215, 0],
 [35936398234, 0],
 [70068684240, 1],
 [80791581268, 1],
 [31207642800, 0],
 [72816082253, 0],
 [19672705204, 0],
 [47655640963, 0],
 [81428197249, 1],
 [65406915720, 0],
 [34979824234, 0],
 [47837250206, 1],
 [55338615720, 0],
 [19631642291, 0],
 [21711615234, 0],
 [50791524272, 0],
 [50810006200, 0],
 [35954590249, 0],
 [42062160291, 0],
 [43403956253, 0]]

### 4. Resumindo os achados


In [51]:
df_resultado = pd.DataFrame.from_records(respostas)
df_resultado.columns = ['cpf', 'resultado']

In [52]:
df_resultado = df_resultado[df_resultado['resultado'] == 1].merge(df_milionarios, how = 'inner', left_on = 'cpf', right_on = 'NR_CPF_CANDIDATO')

In [53]:
df_resultado

,cpf,resultado,SQ_CANDIDATO,VR_BEM_CANDIDATO,NR_CPF_CANDIDATO,NM_CANDIDATO,SG_PARTIDO,SG_UF,NM_UE,DS_CARGO
0,70068684240,1,10000701046,1120000.0,70068684240,NESTOR DIEGO VACA PONZ,DEM,AC,RIO BRANCO,VEREADOR
1,80791581268,1,10000702288,1300000.0,80791581268,WANDERLI PORTELA DE SOUZA,PSD,AC,ACRELÂNDIA,VEREADOR
2,81428197249,1,10000854409,1000000.0,81428197249,MARIA VANEIDA DOS SANTOS SILVA,PP,AC,ACRELÂNDIA,VEREADOR
3,47837250206,1,10000882781,1223000.0,47837250206,JOSMAR GOMES COELHO,PSDB,AC,BRASILÉIA,VEREADOR


A tabela acima resume os casos em que o CPF do milionário aparece dentre os beneficiários do auxílio emergencial.